In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# 10.1 GroupBy机制

In [4]:
df = DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
               'key2': ['one', 'two', 'one', 'two', 'one'],
               'data1': np.random.randn(5),
               'data2': np.random.randn(5)})

In [5]:
df

,key1,key2,data1,data2
0,a,one,-0.110429,0.911974
1,a,two,-0.921336,-0.539582
2,b,one,0.222954,0.149309
3,b,two,0.023444,0.142679
4,a,one,2.540508,-1.886293


In [7]:
grouped = df['data1'].groupby(df['key1'])   #取data1这一列的值并且将key1当作分组依据进行分组

In [8]:
grouped

In [9]:
grouped.mean()   #计算分组平均值

key1
a    0.502914
b    0.123199
Name: data1, dtype: float64

In [10]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [11]:
means

key1  key2
a     one     1.215040
      two    -0.921336
b     one     0.222954
      two     0.023444
Name: data1, dtype: float64

In [12]:
means.unstack()

key2,one,two
key1,,
a,1.215040,-0.921336
b,0.222954,0.023444


In [13]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [14]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [15]:
df['data1'].groupby([states, years]).mean()   #分组键可以是任何长度适当的数组

California  2005   -0.921336
            2006    0.222954
Ohio        2005   -0.043492
            2006    2.540508
Name: data1, dtype: float64

In [16]:
df.groupby('key1').mean()   #默认情况下所有数值列都会被聚合（而key2列不是数值列，被从结果中排除了）

,data1,data2
key1,,
a,0.502914,-0.504634
b,0.123199,0.145994


In [17]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.215040 -0.487159
     two  -0.921336 -0.539582
b    one   0.222954  0.149309
     two   0.023444  0.142679

In [18]:
df.groupby(['key1', 'key2']).size()  #返回分组大小

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

# 对分组进行迭代

In [19]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.110429  0.911974
1    a  two -0.921336 -0.539582
4    a  one  2.540508 -1.886293
b
  key1 key2     data1     data2
2    b  one  0.222954  0.149309
3    b  two  0.023444  0.142679


In [20]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.110429  0.911974
4    a  one  2.540508 -1.886293
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.921336 -0.539582
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.222954  0.149309
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.023444  0.142679


In [21]:
pieces = dict(list(df.groupby('key1')))

In [22]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.222954,0.149309
3,b,two,0.023444,0.142679


In [23]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-0.110429,0.911974
1,a,two,-0.921336,-0.539582
4,a,one,2.540508,-1.886293


In [24]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [26]:
grouped = df.groupby(df.dtypes, axis=1)  #根据dtype对列进行分组

In [27]:
grouped

In [29]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.110429  0.911974
1 -0.921336 -0.539582
2  0.222954  0.149309
3  0.023444  0.142679
4  2.540508 -1.886293
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


# 选取一列或列的子集

In [30]:
df.groupby('key1')['data1']

In [31]:
df.groupby('key1')[['data2']]

In [32]:
df.groupby(['key1', 'key2'])[['data2']].mean()   #只计算data2的平均值并且以DataFrame的形式得到结果

data2
key1 key2          
a    one  -0.487159
     two  -0.539582
b    one   0.149309
     two   0.142679

In [33]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [34]:
s_grouped

In [35]:
s_grouped.mean()

key1  key2
a     one    -0.487159
      two    -0.539582
b     one     0.149309
      two     0.142679
Name: data2, dtype: float64

In [36]:
type(s_grouped.mean())   #得到的结果是Series

pandas.core.series.Series

# 通过字典或Series进行分组

In [37]:
people = DataFrame(np.random.randn(5,5),
                  columns=['a', 'b', 'c', 'd', 'e'],
                  index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [38]:
people

,a,b,c,d,e
Joe,-0.141681,-0.084740,-1.110392,0.189849,0.958444
Steve,0.377546,-0.142239,0.846863,0.446851,0.724362
Wes,0.717655,-0.967083,-0.541055,-1.163538,-0.718852
Jim,0.338183,2.562236,-0.794538,0.631954,0.393149
Travis,0.723318,0.362844,1.034913,1.298582,-1.252052


In [39]:
people.iloc[2:3,[1,2]]=np.nan   #二号行（亦即第三行）1、2位置（亦即第2、3列）数值修改为NaN

In [40]:
people

,a,b,c,d,e
Joe,-0.141681,-0.084740,-1.110392,0.189849,0.958444
Steve,0.377546,-0.142239,0.846863,0.446851,0.724362
Wes,0.717655,NaN,NaN,-1.163538,-0.718852
Jim,0.338183,2.562236,-0.794538,0.631954,0.393149
Travis,0.723318,0.362844,1.034913,1.298582,-1.252052


In [41]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [42]:
by_column = people.groupby(mapping, axis=1)   #直接传递字典来构造数组

In [43]:
by_column.sum()

,blue,red
Joe,-0.920543,0.732023
Steve,1.293714,0.959669
Wes,-1.163538,-0.001197
Jim,-0.162584,3.293568
Travis,2.333495,-0.165891


In [44]:
map_series = Series(mapping)

In [45]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [46]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


# 通过函数进行分组

In [47]:
people.groupby(len).sum()  #按照名字的长度进行分组统计（分别是字母为3个、5个、6个的长度）

,a,b,c,d,e
3,0.914157,2.477496,-1.904930,-0.341735,0.632741
5,0.377546,-0.142239,0.846863,0.446851,0.724362
6,0.723318,0.362844,1.034913,1.298582,-1.252052


In [48]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [49]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.141681 -0.084740 -1.110392 -1.163538 -0.718852
  two  0.338183  2.562236 -0.794538  0.631954  0.393149
5 one  0.377546 -0.142239  0.846863  0.446851  0.724362
6 two  0.723318  0.362844  1.034913  1.298582 -1.252052

# 根据索引级别分组

In [50]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                   names=['cty', 'tenor'])

In [51]:
hier_df = DataFrame(np.random.randn(4,5), columns=columns)

In [52]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.002282 -0.567665 -0.942041  1.416226  0.422403
1      0.893530 -0.554476 -0.190773  1.214119 -0.042945
2      1.014643 -1.357321  0.851232  2.065421  0.682704
3     -0.986100 -0.620368 -1.740568 -0.176899  1.374512

In [53]:
hier_df.groupby(level='cty', axis=1).count() #根据级别分组要用到level关键字

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 数据聚合

In [54]:
df

,key1,key2,data1,data2
0,a,one,-0.110429,0.911974
1,a,two,-0.921336,-0.539582
2,b,one,0.222954,0.149309
3,b,two,0.023444,0.142679
4,a,one,2.540508,-1.886293


In [55]:
grouped = df.groupby('key1')

In [56]:
grouped['data1'].quantile(0.9)

key1
a    2.010321
b    0.203003
Name: data1, dtype: float64

In [59]:
def peak_to_peak(arr):
    return arr.max()-arr.min()

In [60]:
grouped.agg(peak_to_peak)  #使用自己的聚合函数通过aggregate或者agg方法即可

,data1,data2
key1,,
a,3.461844,2.798268
b,0.199510,0.006630


In [61]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.502914  1.810589 -0.921336 -0.515882 -0.110429  1.215040   
b      2.0  0.123199  0.141075  0.023444  0.073322  0.123199  0.173077   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     2.540508   3.0 -0.504634  1.399461 -1.886293 -1.212938 -0.539582   
b     0.222954   2.0  0.145994  0.004688  0.142679  0.144337  0.145994   

                          
           75%       max  
key1                      
a     0.186196  0.911974  
b     0.147652  0.149309

# 面向列的多函数应用

In [63]:
tips = pd.read_csv(r'C:\Users\zoe_x\Documents\python\pydata-book-2nd-edition\examples\tips.csv')

In [64]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [65]:
tips['tip_pct']=tips['tip']/tips['total_bill']

In [66]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [67]:
grouped = tips.groupby(['day', 'smoker'])  #根据天和smoker对tips进行分组

In [68]:
grouped_pct = grouped['tip_pct']

In [69]:
grouped_pct.agg('mean')   #对分组结果取平均值

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [70]:
grouped_pct.agg(['mean', 'std', peak_to_peak])   #对分组后的结果求多个函数结果，并且按照函数名称来命名相应的列

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [71]:
grouped_pct.agg([('foo', 'mean'),('bar', np.std)])  #如果传入的是（name,function）元组组成的列表，那么列命名就按照传入元组的第一个元素

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [72]:
functions = ['count', 'mean', 'max']

In [73]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [74]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [75]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [76]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [77]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [78]:
grouped.agg({'tip' : np.max, 'size' :'sum'})   #对不同的列应用不同的函数,可以传入字典

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [79]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
            'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

# 以“没有行索引”的形式返回聚合函数

In [80]:
tips.groupby(['day', 'smoker'], as_index=False).mean()   #as_index=False禁用的是使用唯一的分组键组成索引

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [81]:
tips.groupby(['day', 'smoker'], as_index=True).mean()   #和上面对比一下

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

# 10.3 apply：一般性的“拆分-应用-合并”

In [82]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [84]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [86]:
tips.groupby('smoker').apply(top)   #对smoker分组并用函数调用apply，得到分组后选出的最高的5个tip_pct值

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [87]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')  #分组后取分组结果中total_bill最大的一个（将参数放在函数名后面一并传入）

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [88]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [89]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [91]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

# 禁止分组键

In [92]:
tips.groupby('smoker', group_keys=False).apply(top)   #禁止分组键和原始对象的索引共同构成结果对象中的层次化索引

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [94]:
tips.groupby('smoker').apply(top)   #和上面对比

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

# 分位数和桶分析

In [95]:
frame = DataFrame({'data1': np.random.randn(1000),
                  'data2': np.random.randn(1000)})

In [99]:
frame.head()

,data1,data2
0,1.801640,1.323621
1,0.170839,0.043858
2,0.803708,0.892626
3,-0.537848,-0.955070
4,1.471821,-0.035326


In [96]:
quartiles = pd.cut(frame.data1, 4)  #按照data1列的数值大小划分四个区间，并将每个值所属的区间表示出来

In [97]:
quartiles[:10]  #前十行

0       (0.32, 2.102]
1      (-1.462, 0.32]
2       (0.32, 2.102]
3      (-1.462, 0.32]
4       (0.32, 2.102]
5      (-1.462, 0.32]
6      (-1.462, 0.32]
7    (-3.251, -1.462]
8       (0.32, 2.102]
9       (0.32, 2.102]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.251, -1.462] < (-1.462, 0.32] < (0.32, 2.102] < (2.102, 3.884]]

In [100]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean' : group.mean()}

In [101]:
grouped = frame.data2.groupby(quartiles)  #按照区间进行分组

In [102]:
grouped.apply(get_stats)

data1                  
(-3.251, -1.462]  count     61.000000
                  max        2.454686
                  mean       0.112080
                  min       -2.195231
(-1.462, 0.32]    count    555.000000
                  max        2.911468
                  mean      -0.004092
                  min       -3.347944
(0.32, 2.102]     count    370.000000
                  max        2.285778
                  mean       0.000113
                  min       -2.871426
(2.102, 3.884]    count     14.000000
                  max        1.800738
                  mean       0.358123
                  min       -0.569057
Name: data2, dtype: float64

In [103]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.251, -1.462]",61.0,2.454686,0.112080,-2.195231
"(-1.462, 0.32]",555.0,2.911468,-0.004092,-3.347944
"(0.32, 2.102]",370.0,2.285778,0.000113,-2.871426
"(2.102, 3.884]",14.0,1.800738,0.358123,-0.569057


In [104]:
grouping = pd.qcut(frame.data1, 10, labels=False) #获得大小相等的桶，labels=False就只会获得分位数的编号（如下，0-9）

In [105]:
grouping 

0      9
1      5
2      7
3      2
4      9
5      1
6      0
7      0
8      6
9      6
10     6
11     2
12     1
13     2
14     9
15     2
16     0
17     7
18     3
19     8
20     5
21     3
22     6
23     5
24     6
25     2
26     4
27     5
28     2
29     6
      ..
970    6
971    5
972    3
973    4
974    5
975    7
976    2
977    9
978    9
979    3
980    1
981    2
982    2
983    8
984    6
985    2
986    9
987    1
988    3
989    2
990    6
991    0
992    8
993    6
994    2
995    0
996    3
997    8
998    7
999    8
Name: data1, Length: 1000, dtype: int64

In [106]:
grouped = frame.data2.groupby(grouping)

In [107]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.454686,0.063717,-2.665294
1,100.0,2.324201,-0.052320,-3.109437
2,100.0,2.605650,0.045407,-2.635940
3,100.0,2.911468,0.106932,-2.887174
4,100.0,2.241382,-0.063885,-2.593644
5,100.0,2.113264,-0.009738,-3.347944
6,100.0,2.209738,0.008401,-2.125645
7,100.0,2.285778,-0.053641,-2.871426
8,100.0,2.050846,0.072044,-2.326441


# 示例：用特定于分组的值填充缺失值

In [111]:
s = Series(np.random.randn(6))

In [112]:
s

0   -0.150058
1    0.007184
2    0.223167
3   -1.147883
4   -1.306311
5   -1.586107
dtype: float64

In [113]:
s[::2]=np.nan

In [114]:
s

0         NaN
1    0.007184
2         NaN
3   -1.147883
4         NaN
5   -1.586107
dtype: float64

In [115]:
s.fillna(s.mean()) #用均值填充

0   -0.908935
1    0.007184
2   -0.908935
3   -1.147883
4   -0.908935
5   -1.586107
dtype: float64

In [116]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
         'Oregon', 'Nevada', 'California', 'Idaho']

In [117]:
group_key = ['East'] * 4 + ['West'] * 4  #形成一个元素为八个的list

In [118]:
data = Series(np.random.randn(8), index=states)

In [119]:
data

Ohio          0.108740
New York      0.622464
Vermont      -0.237474
Florida      -1.841507
Oregon        0.840519
Nevada        0.113462
California   -1.247346
Idaho         0.378019
dtype: float64

In [121]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [122]:
data

Ohio          0.108740
New York      0.622464
Vermont            NaN
Florida      -1.841507
Oregon        0.840519
Nevada             NaN
California   -1.247346
Idaho              NaN
dtype: float64

In [123]:
data.groupby(group_key).mean()

East   -0.370101
West   -0.203413
dtype: float64

In [124]:
fill_mean = lambda g: g.fillna(g.mean())

In [125]:
data.groupby(group_key).apply(fill_mean) #用分组平均值填充NaN

Ohio          0.108740
New York      0.622464
Vermont      -0.370101
Florida      -1.841507
Oregon        0.840519
Nevada       -0.203413
California   -1.247346
Idaho        -0.203413
dtype: float64

In [126]:
fill_values = {'East': 0.5, 'West': -1}  #预定义各组的填充值

In [127]:
fill_func =lambda g: g.fillna(fill_values[g.name])

In [129]:
data.groupby(group_key).apply(fill_func)

Ohio          0.108740
New York      0.622464
Vermont       0.500000
Florida      -1.841507
Oregon        0.840519
Nevada       -1.000000
California   -1.247346
Idaho        -1.000000
dtype: float64

# 示例：随机采样和排列

In [130]:
suits = ['H', 'S', 'C', 'D']

In [131]:
card_val = (list(range(1,11)) + [10] * 3) * 4

In [134]:
base_names=['A'] + list(range(2,11)) + ['J', 'K', 'Q']

In [135]:
cards = []

In [136]:
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

In [138]:
deck = Series(card_val, index=cards)

In [139]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [140]:
def draw(deck, n=5):
    return deck.sample(n)

In [141]:
draw(deck)   #随机抽取5个

KD    10
7H     7
5C     5
KH    10
6S     6
dtype: int64

In [142]:
get_suit = lambda card: card[-1]  #按照最后一个字母分组

In [143]:
deck.groupby(get_suit).apply(draw, n=2)

C  6C     6
   3C     3
D  KD    10
   AD     1
H  KH    10
   2H     2
S  QS    10
   4S     4
dtype: int64

In [144]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)  #和上面是等价的，但是没有层次化索引

5C     5
KC    10
8D     8
6D     6
5H     5
8H     8
6S     6
2S     2
dtype: int64

# 示例：分组加权平均数和相关系数

In [145]:
df = DataFrame({'category': ['a', 'a', 'a', 'a',
                            'b', 'b', 'b', 'b'],
               'data' : np.random.randn(8),
               'weights' : np.random.randn(8)})

In [146]:
df

,category,data,weights
0,a,-0.570105,-0.080796
1,a,0.695549,-0.532739
2,a,-0.369197,-0.357282
3,a,0.791790,-0.420310
4,b,-0.037048,0.559006
5,b,-0.864601,-0.062830
6,b,1.654672,-0.185990
7,b,-0.807136,-0.589563


In [147]:
grouped = df.groupby('category')

In [148]:
get_wavg = lambda g: np.average(g['data'], weights = g['weights'])

In [149]:
grouped.apply(get_wavg)  #用category分组并计算加权平均数

category
a    0.37766
b   -0.72203
dtype: float64

In [150]:
close_px = pd.read_csv(r'C:\Users\zoe_x\Documents\python\pydata-book-2nd-edition\examples\stock_px_2.csv', parse_dates=True, index_col=0)

In [151]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [153]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [154]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [155]:
spx_corr = lambda x: x.corrwith(x['SPX']) #创建函数，用来计算每列和SPX列的成对相关系数

In [156]:
rets = close_px.pct_change().dropna() #计算百分比变化（舍去NaN值）

In [157]:
get_year = lambda x: x.year  #用年对百分比分组

In [159]:
by_year = rets.groupby(get_year)

In [160]:
by_year.apply(spx_corr)  #按年返回相关系数

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [161]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 示例：组级别的线性回归

In [169]:
import statsmodels.api as sm   #引入库
def regress(data, yvar, xvars):
    Y=data[yvar]
    X=data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y,X).fit()
    return result.params

In [170]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 透视表和交叉表

In [172]:
tips.pivot_table(index=['day', 'smoker']) #根据day和smoker计算分组平均数，并且将day和smoker放到行上

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [173]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],   #只想聚合tip_pct和size，并且根据time进行分组，将smoker放到列上，day放到行上
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [174]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],   #添加分项小计(all为平均数)
                columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [175]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',    #得到有关分组大小的交叉表（计数或频率）
                aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [176]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],   #NaN填充为0
                columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

# 交叉表：crosstab

In [177]:
data

Ohio          0.108740
New York      0.622464
Vermont            NaN
Florida      -1.841507
Oregon        0.840519
Nevada             NaN
California   -1.247346
Idaho              NaN
dtype: float64

In [ ]:
# 具体见书